In [28]:
import polars as pl
import pickle as pkl
from backtest_lib.market.polars_impl import PolarsPastView
from backtest_lib.market import PastView

close_df = pkl.load(open("./sp500_close.pkl", "rb"))
close_df = close_df.dropna(axis=1, how="any")
dates = close_df.index.values

close_pl = pl.from_pandas(close_df)
securities = close_pl.columns
close_prices_df = close_pl.with_columns(pl.Series("date", dates))
past_cost_prices = PolarsPastView.from_data_frame(close_prices_df)

past_cost_prices

PolarsPastView(by_period=PolarsByPeriod(_period_column_df=shape: (491, 1_256)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ 2020-08-3 ┆ 2020-09-0 ┆ 2020-09-0 ┆ 2020-09-0 ┆ … ┆ 2025-08-2 ┆ 2025-08-2 ┆ 2025-08-2 ┆ 2025-08- │
│ 1 00:00:0 ┆ 1 00:00:0 ┆ 2 00:00:0 ┆ 3 00:00:0 ┆   ┆ 6 00:00:0 ┆ 7 00:00:0 ┆ 8 00:00:0 ┆ 29 00:00 │
│ 0.0000000 ┆ 0.0000000 ┆ 0.0000000 ┆ 0.0000000 ┆   ┆ 0.0000000 ┆ 0.0000000 ┆ 0.0000000 ┆ :00.0000 │
│ 00        ┆ 00        ┆ 00        ┆ 00        ┆   ┆ 00        ┆ 00        ┆ 00        ┆ 00000    │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 31.540001 ┆ 31.6      ┆ 32.130001 ┆ 32.119999 ┆ … ┆ 90.900002 ┆ 91.709999 ┆ 91.199997 ┆ 91.52999 │
│           ┆

In [29]:
from backtest_lib.market.polars_impl import PolarsPastView
from backtest_lib.market import MarketView, PastUniversePrices

df = past_cost_prices.by_security.as_df()
cols = df.columns
ewma = (
    df.with_columns(pl.col(col).ewm_mean(com=2).alias(f"{col}_EWM") for col in cols)
    .with_columns((pl.col(col) / pl.col(f"{col}_EWM")) for col in cols)
    .select(cols)
    .with_columns(date=dates)
)

ewma_past_view = PolarsPastView.from_data_frame(ewma)

market_view = MarketView(
    prices=PastUniversePrices(close=past_cost_prices),
    signals={"ewma_ratio": ewma_past_view},
    periods=dates,
)

market_view.signals["ewma_ratio"].by_period[-1]

SeriesUniverseMapping(names=('ACGL', 'AMD', 'ANET', 'AKAM', 'ALGN', 'ABBV', 'AMGN', 'ADSK', 'ACN', 'A', 'AIZ', 'AWK', 'AMCR', 'AMAT', 'APH', 'ALLE', 'ALL', 'ARE', 'APO', 'AAPL', 'ALB', 'GOOG', 'AMT', 'AME', 'AES', 'AFL', 'APA', 'APD', 'AON', 'ADM', 'MMM', 'APTV', 'ADBE', 'AMP', 'ADI', 'AMZN', 'T', 'AJG', 'AEP', 'GOOGL', 'AXP', 'AEE', 'AIG', 'AOS', 'LNT', 'ATO', 'MO', 'ABT', 'ADP', 'CRL', 'BLDR', 'BSX', 'BKNG', 'BIIB', 'AZO', 'CHTR', 'BRK-B', 'CZR', 'BKR', 'CF', 'BLK', 'BBY', 'BDX', 'BAX', 'CBRE', 'CARR', 'CHRW', 'CPT', 'BG', 'BXP', 'CNP', 'SCHW', 'BAC', 'BALL', 'BA', 'TECH', 'COR', 'CPB', 'CVX', 'BK', 'CCL', 'BMY', 'KMX', 'BF-B', 'XYZ', 'AVGO', 'CDNS', 'BX', 'CBOE', 'CAH', 'CNC', 'AXON', 'BR', 'BRO', 'CDW', 'AVY', 'AVB', 'COF', 'CAT', 'DVA', 'CPAY', 'DDOG', 'CSCO', 'DLTR', 'DECK', 'DAY', 'STZ', 'DLR', 'COST', 'CRWD', 'CFG', 'DAL', 'CTVA', 'DG', 'DVN', 'CSX', 'DELL', 'CMI', 'CB', 'GLW', 'C', 'CAG', 'CL', 'CTAS', 'CTRA', 'DHR', 'KO', 'CME', 'CINF', 'CMS', 'CI', 'CHD', 'CSGP', 'FANG', 'DX

In [30]:
from backtest_lib.portfolio import WeightedPortfolio
from backtest_lib.strategy import Decision


def ewma_strategy(universe, current_portfolio, market, ctx):
    latest_ewma_ratio = (
        market.signals["ewma_ratio"].by_security[list(universe)].by_period[-1]
    )

    avg_ewma_ratio = latest_ewma_ratio.mean()
    norm_ewma_ratio = latest_ewma_ratio / avg_ewma_ratio
    weights = norm_ewma_ratio / len(norm_ewma_ratio)
    return Decision(target=WeightedPortfolio(cash=0, holdings=weights))


decision = ewma_strategy(["AAPL", "MSFT"], None, market_view, None)
decision

Decision(target=WeightedPortfolio(cash=0, holdings=SeriesUniverseMapping(names=('AAPL', 'MSFT'), _data=shape: (2,)
Series: '2025-08-29 00:00:00.000000000' [f64]
[
	0.501648
	0.498352
], _scalar_type=<class 'float'>)))

In [31]:
from backtest_lib.backtest import Backtest
from backtest_lib.market.polars_impl import SeriesUniverseMapping
import polars as pl

universe = tuple(past_cost_prices.by_security.as_df().columns)
pf = WeightedPortfolio(
    cash=0,
    holdings=SeriesUniverseMapping.from_names_and_data(
        universe, pl.Series([1 / len(universe)] * len(universe))
    ),
)
backtest = Backtest(
    universe=universe,
    strategy=ewma_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

results = backtest.run()
print(f"total return: {(results.total_growth - 1) * 100:.2f}%")

total return: 150.77%


In [32]:
def index_strategy(universe, current_portfolio, market, ctx) -> Decision:
    return Decision(pf)


benchmark_backtest = Backtest(
    universe=universe,
    strategy=index_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

results = benchmark_backtest.run()
print(f"total return: {(results.total_growth - 1) * 100:.2f}%")

total return: 95.06%


In [33]:
import polars as pl

first_prices = past_cost_prices.by_period[0]
last_prices = past_cost_prices.by_period[-1]

avg_first_change = (
    past_cost_prices.by_period[1] / past_cost_prices.by_period[0]
).mean()

print("avg_first_change:", avg_first_change)

(last_prices / first_prices).mean()

avg_first_change: 1.0043851702912916


1.9677977477606803

In [34]:
aapl_prices = past_cost_prices.by_security["AAPL"].as_series()

print(aapl_prices.pct_change() + 1)
print((aapl_prices.pct_change() + 1).cum_prod())

shape: (1_256,)
Series: 'AAPL' [f64]
[
	null
	1.039833
	0.979282
	0.919939
	1.000662
	…
	0.997366
	1.009465
	1.005146
	1.008981
	0.998194
]
shape: (1_256,)
Series: 'AAPL' [f64]
[
	null
	1.039833
	1.018289
	0.936764
	0.937384
	…
	1.760384
	1.777046
	1.78619
	1.802232
	1.798977
]


In [35]:
past_cost_prices.by_period[-1].mean()

229.23348196460856